In [2]:
import os
import pandas as pd

import torch
import torchaudio

In [3]:
positions = ['AV', 'MV', 'TV', 'PV']

patients_df = pd.read_csv('dataset/data.csv')
patients_df.head()

,Unnamed: 0,id,AV,PV,TV,MV,Phc,Age,Sex,Height,...,Systolic murmur pitch,Systolic murmur quality,Diastolic murmur timing,Diastolic murmur shape,Diastolic murmur grading,Diastolic murmur pitch,Diastolic murmur quality,Outcome,Campaign,Additional ID
0,0,13918,True,True,True,True,False,Child,Male,98.0,...,Low,Blowing,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,NaN
1,1,14241,True,True,True,True,False,Child,Male,87.0,...,Low,Harsh,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,NaN
2,2,14998,True,True,True,True,False,Child,Male,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,NaN
3,3,23625,True,True,True,True,False,Child,Female,92.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,50379.0
4,4,24160,True,True,True,True,False,Child,Female,98.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,NaN


In [4]:
label_df = pd.read_csv('dataset/training_data/13918_AV.tsv', delimiter='\t', header=None)
label_df.head()

,0,1,2
0,0.000000,1.146750,0
1,1.146750,1.300191,1
2,1.300191,1.400191,2
3,1.400191,1.540191,3
4,1.540191,1.779916,4


In [5]:
import csv
import glob

# 각 음성 데이터별 s1, s2 소리 길이의 평균과 표준편차 구하기
f = open('s1_s2_length.csv', 'w', newline='')
writer = csv.writer(f)
writer.writerow(['filename', 's1_count', 's1_mean', 's1_std', 's2_count', 's2_mean', 's2_std'])

tsv_paths = glob.glob('./dataset/training_data/*.tsv')

for tsv_path in tsv_paths:
    tsv_filename = os.path.abspath(tsv_path)
    label_df = pd.read_csv(tsv_filename, delimiter='\t', header=None)
    s1_df = label_df[label_df[2] == 1]
    s2_df = label_df[label_df[2] == 3]
    s1_length_s = s1_df[1] - s1_df[0]
    s2_length_s = s2_df[1] - s2_df[0]

    s1_count = s1_length_s.count()
    s1_mean = s1_length_s.mean()
    s1_std = s1_length_s.std()
    s2_count = s2_length_s.count()
    s2_mean = s2_length_s.mean()
    s2_std = s2_length_s.std()
    writer.writerow([os.path.basename(tsv_path)[:-4], s1_count, s1_mean, s1_std, s2_count, s2_mean, s2_std])
            
f.close()

In [6]:
s1s2_length_df = pd.read_csv('s1_s2_length.csv')
s1s2_length_df.head()

,filename,s1_count,s1_mean,s1_std,s2_count,s2_mean,s2_std
0,13918_AV,15,0.147792,0.017113,15,0.124115,0.018223
1,13918_MV,11,0.101109,0.013703,10,0.078204,0.004823
2,13918_PV,9,0.111568,0.009613,9,0.103588,0.013194
3,13918_TV,10,0.125168,0.015879,10,0.116000,0.015776
4,14241_AV,13,0.106689,0.019819,13,0.095834,0.013956


In [7]:
s1_length_total_mean = (s1s2_length_df['s1_mean'] * s1s2_length_df['s1_count']).sum() / s1s2_length_df['s1_count'].sum()
s2_length_total_mean = (s1s2_length_df['s2_mean'] * s1s2_length_df['s2_count']).sum() / s1s2_length_df['s2_count'].sum()

print('s1 mean:', s1_length_total_mean, 's2 mean:', s2_length_total_mean)

s1 mean: 0.11916449658236496 s2 mean: 0.10533918449571063


In [24]:
import math

s1_expect_x_square = ((s1s2_length_df['s1_std'] ** 2 + s1s2_length_df['s1_mean'] ** 2) * s1s2_length_df['s1_count']).sum() / s1s2_length_df['s1_count'].sum()
s2_expect_x_square = ((s1s2_length_df['s2_std'] ** 2 + s1s2_length_df['s2_mean'] ** 2) * s1s2_length_df['s2_count']).sum() / s1s2_length_df['s2_count'].sum()

s1_expect_x = (s1s2_length_df['s1_mean'] * s1s2_length_df['s1_count']).sum() / s1s2_length_df['s1_count'].sum()
s2_expect_x = (s1s2_length_df['s2_mean'] * s1s2_length_df['s2_count']).sum() / s1s2_length_df['s2_count'].sum()

s1_x_total_std = math.sqrt(s1_expect_x_square - s1_expect_x ** 2)
s2_x_total_std = math.sqrt(s2_expect_x_square - s2_expect_x ** 2)

print('s1 std:', s1_x_total_std, 's2 std:', s2_x_total_std)

s1 std: 0.024126766466607278 s2 std: 0.02370798587163986
